In [3]:
# Imports (keep minimal)
import pandas as pd
import numpy as np
from src.extract_dataframe import (
    get_open_data_urls_and_game,
    make_tracking_df,
    make_players_df,
)
from src.utils import (
    time_to_seconds,
    get_first_possession_in_phase,
    get_all_possessions_in_phase,
    add_possession_ids_to_phases_from_events,
    enrich_events_with_linked_events,
    add_xthreat_at_start_for_possessions,
    add_xthreat_at_end_for_possessions,
    add_xthreat_potential_max,
    add_passing_option_proximity,
    add_possession_proximity,
    add_xthreat_deltas,
    add_avg_po_proximity_score
)
pd.set_option('display.max_columns', 100)

In [10]:
# Parameters
match_id = 1886347

# Use helper to get URLs and game object
meta_data_github_url, tracking_data_github_url, dynamic_events_data_github_url, phase_data_github_url, game = get_open_data_urls_and_game(
    match_id=match_id,
    only_alive=False,
)
# # Add velocities on tracking_data (databallpy)
# game.tracking_data.add_velocity(column_ids=game.get_column_ids())

In [ ]:
# Build tracking_df from Open Data tracking JSONL
tracking_df = make_tracking_df(tracking_data_github_url, match_id)
tracking_df.head()

,x,y,player_id,is_detected,frame,timestamp,period,possession_player_id,possession_group,ball_x,ball_y,ball_z,is_detected_ball,match_id
0,-39.63,-0.08,51009,False,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347
1,-19.21,-9.18,176224,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347
2,-21.83,0.47,51649,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347
3,-1.16,-32.47,50983,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347
4,-18.88,15.73,735578,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347


In [ ]:
# Build players_df from match metadata JSON
players_df = make_players_df(meta_data_github_url)
players_df.head()

,start_time,end_time,match_name,date_time,home_team.name,away_team.name,id,short_name,number,team_id,team_name,player_role.position_group,total_time,player_role.name,player_role.acronym,is_gk,direction_player_1st_half,direction_player_2nd_half
0,00:00:00,01:25:21,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,38673,G. May,10,4177,Auckland FC,Center Forward,5121,Center Forward,CF,False,right_to_left,left_to_right
1,00:00:00,None,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,51713,C. Elliott,17,4177,Auckland FC,Full Back,5400,Right Back,RB,False,right_to_left,left_to_right
2,00:00:00,01:16:37,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,50951,J. Brimmer,22,4177,Auckland FC,Center Forward,4597,Center Forward,CF,False,right_to_left,left_to_right
3,00:00:00,01:24:58,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,50978,C. Timmins,19,1805,Newcastle United Jets FC,Midfield,5098,Left Defensive Midfield,LDM,False,left_to_right,right_to_left
4,00:00:00,None,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,133498,F. De Vries,15,4177,Auckland FC,Full Back,5400,Left Back,LB,False,right_to_left,left_to_right


In [7]:
# Merge tracking and players to create enriched_tracking_data
enriched_tracking_data = tracking_df.merge(players_df, left_on=['player_id'], right_on=['id'])
enriched_tracking_data.head()

,x,y,player_id,is_detected,frame,timestamp,period,possession_player_id,possession_group,ball_x,ball_y,ball_z,is_detected_ball,match_id,start_time,end_time,match_name,date_time,home_team.name,away_team.name,id,short_name,number,team_id,team_name,player_role.position_group,total_time,player_role.name,player_role.acronym,is_gk,direction_player_1st_half,direction_player_2nd_half
0,-39.63,-0.08,51009,False,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347,00:00:00,None,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,51009,R. Scott,1,1805,Newcastle United Jets FC,Other,5400,Goalkeeper,GK,True,left_to_right,right_to_left
1,-19.21,-9.18,176224,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347,00:00:00,None,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,176224,P. Cancar,4,1805,Newcastle United Jets FC,Central Defender,5400,Right Center Back,RCB,False,left_to_right,right_to_left
2,-21.83,0.47,51649,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347,00:00:00,None,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,51649,A. Šušnjar,15,1805,Newcastle United Jets FC,Central Defender,5400,Left Center Back,LCB,False,left_to_right,right_to_left
3,-1.16,-32.47,50983,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347,00:00:00,None,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,50983,D. Ingham,14,1805,Newcastle United Jets FC,Full Back,5400,Right Back,RB,False,left_to_right,right_to_left
4,-18.88,15.73,735578,True,10,2025-12-29,1.0,NaN,None,0.32,0.38,0.13,True,1886347,00:00:00,None,Auckland FC vs Newcastle United Jets FC,2024-11-30T04:00:00Z,Auckland FC,Newcastle United Jets FC,735578,M. Natta,33,1805,Newcastle United Jets FC,Full Back,5400,Left Back,LB,False,left_to_right,right_to_left


In [ ]:
# Load dynamic events for the match
de_match = pd.read_csv(dynamic_events_data_github_url)
de_match.head()

,event_id,index,match_id,frame_start,frame_end,frame_physical_start,time_start,time_end,minute_start,second_start,duration,period,attacking_side_id,attacking_side,event_type_id,event_type,event_subtype_id,event_subtype,player_id,player_name,player_position_id,player_position,player_in_possession_id,player_in_possession_name,player_in_possession_position_id,player_in_possession_position,team_id,team_shortname,x_start,y_start,channel_id_start,channel_start,third_id_start,third_start,penalty_area_start,x_end,y_end,channel_id_end,channel_end,third_id_end,third_end,penalty_area_end,associated_player_possession_event_id,associated_player_possession_frame_start,associated_player_possession_frame_end,associated_player_possession_end_type_id,associated_player_possession_end_type,associated_off_ball_run_event_id,associated_off_ball_run_subtype_id,associated_off_ball_run_subtype,...,xthreat,xpass_completion,passing_option_score,predicted_passing_option,peak_passing_option_frame,passing_option_at_player_possession_start,n_simultaneous_runs,give_and_go,intended_run_behind,push_defensive_line,break_defensive_line,passing_option_at_start,n_simultaneous_passing_options,passing_option_at_pass_moment,n_opponents_ahead_end,n_opponents_ahead_start,n_opponents_overtaken,pressing_chain,pressing_chain_length,pressing_chain_end_type_id,pressing_chain_end_type,pressing_chain_index,index_in_pressing_chain,simultaneous_defensive_engagement_same_target,simultaneous_defensive_engagement_same_target_rank,affected_line_breaking_passing_option_id,affected_line_break_id,affected_line_break,affected_line_breaking_passing_option_attempted,affected_line_breaking_passing_option_xthreat,affected_line_breaking_passing_option_dangerous,affected_line_breaking_passing_option_run_subtype_id,affected_line_breaking_passing_option_run_subtype,possession_danger,beaten_by_possession,beaten_by_movement,stop_possession_danger,reduce_possession_danger,force_backward,xloss_player_possession_start,xloss_player_possession_end,xloss_player_possession_max,xshot_player_possession_start,xshot_player_possession_end,xshot_player_possession_max,is_player_possession_start_matched,is_player_possession_end_matched,is_previous_pass_matched,is_pass_reception_matched,fully_extrapolated
0,8_0,0,1886347,28,28,NaN,00:01.8,00:01.8,0,1,0.0,1,1,left_to_right,8,player_possession,NaN,NaN,966120,B. Gibson,19,CF,NaN,NaN,NaN,NaN,1805,Newcastle,0.73,0.49,3,center,2,middle_third,False,0.73,0.49,3,center,2,middle_third,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,True,False
1,8_1,1,1886347,48,58,NaN,00:03.8,00:04.8,0,3,1.0,1,1,left_to_right,8,player_possession,NaN,NaN,51649,A. Šušnjar,4,LCB,NaN,NaN,NaN,NaN,1805,Newcastle,-22.31,1.22,3,center,1,defensive_third,False,-22.21,2.70,3,center,1,defensive_third,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,False
2,7_0,2,1886347,48,53,NaN,00:03.8,00:04.3,0,3,0.5,1,1,left_to_right,7,passing_option,NaN,NaN,735574,K. Grozos,11,RDM,51649.0,A. Šušnjar,4.0,LCB,1805,Newcastle,-10.47,-2.78,3,center,2,middle_third,False,-11.36,-1.38,3,center,2,middle_third,False,8_1,48.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0006,0.9629,0.9080,True,49.0,True,NaN,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,False
3,7_1,3,1886347,48,58,NaN,00:03.8,00:04.8,0,3,1.0,1,1,left_to_right,7,passing_option,NaN,NaN,735578,M. Natta,2,LB,51649.0,A. Šušnjar,4.0,LCB,1805,Newcastle,-20.69,16.66,2,half_space_left,1,defensive_third,False,-20.36,17.69,2,half_space_left,1,defensive_third,False,8_1,48.0,NaN,NaN,NaN

In [11]:
# Read phases of play data
phases_match = pd.read_csv(phase_data_github_url)
phases_match.head()

,index,match_id,frame_start,frame_end,time_start,time_end,minute_start,second_start,duration,period,attacking_side_id,team_in_possession_id,attacking_side,team_in_possession_shortname,n_player_possessions_in_phase,team_possession_loss_in_phase,team_possession_lead_to_goal,team_possession_lead_to_shot,team_in_possession_phase_type,team_in_possession_phase_type_id,team_out_of_possession_phase_type,team_out_of_possession_phase_type_id,x_start,y_start,channel_id_start,channel_start,third_id_start,third_start,penalty_area_start,x_end,y_end,channel_id_end,channel_end,third_id_end,third_end,penalty_area_end,team_in_possession_width_start,team_in_possession_width_end,team_in_possession_length_start,team_in_possession_length_end,team_out_of_possession_width_start,team_out_of_possession_width_end,team_out_of_possession_length_start,team_out_of_possession_length_end
0,0,1886347,28,89,00:01.8,00:07.9,0,1,6.1,1,1,1805,left_to_right,Newcastle,3,False,False,False,create,1,medium_block,9,0.03,-0.36,3,center,2,middle_third,False,-10.70,23.82,1,wide_left,2,middle_third,False,51.72,54.41,41.44,58.03,34.37,32.81,42.01,52.35
1,1,1886347,89,107,00:07.9,00:09.7,0,7,1.8,1,1,1805,left_to_right,Newcastle,1,True,False,False,direct,6,defending_direct,15,-9.87,23.99,1,wide_left,2,middle_third,False,21.73,16.53,2,half_space_left,3,attacking_third,False,54.72,54.54,58.24,62.34,32.97,34.26,51.98,46.99
2,2,1886347,185,232,00:17.5,00:22.2,0,17,4.7,1,2,4177,right_to_left,Auckland FC,2,False,False,False,build_up,0,high_block,10,-22.52,-7.98,3,center,1,defensive_third,False,-20.40,17.83,2,half_space_left,1,defensive_third,False,45.30,50.78,48.51,50.31,31.40,36.38,54.72,51.13
3,3,1886347,232,283,00:22.2,00:27.3,0,22,5.1,1,2,4177,right_to_left,Auckland FC,1,True,False,False,create,1,medium_block,9,-19.24,19.52,2,half_space_left,1,defensive_third,False,29.20,26.16,1,wide_left,3,attacking_third,False,50.81,42.83,50.33,63.89,36.59,38.73,51.22,53.46
4,4,1886347,283,301,00:27.3,00:29.1,0,27,1.8,1,1,1805,left_to_right,Newcastle,1,True,False,False,chaotic,5,chaotic,5,-29.73,-26.19,5,wide_right,1,defensive_third,False,-36.35,-29.24,5,wide_right,1,defensive_third,False,38.61,36.66,53.59,54.36,42.62,40.13,64.54,71.83


In [12]:
# --- iterate through all phases in phases_match ---
enriched_phase_rows = []

for _, row in phases_match.iterrows():
    phase_index = int(row["index"])
    frame_start = int(row["frame_start"])
    frame_end = int(row["frame_end"])

    # identify all possession events in THIS phase
    pp_all = get_all_possessions_in_phase(
        de_match,
        phase_frame_start=frame_start,
        phase_frame_end=frame_end,
        phase_index=phase_index,
        enforce_phase_index=False,
        sort=True
    )

    # store them (for debugging or downstream use)
    enriched_phase_rows.append({
        "phase_index": phase_index,
        "frame_start": frame_start,
        "frame_end": frame_end,
        "pp_all": pp_all
    })

# after the loop, build the enriched phases table:
enriched_phases_match = add_possession_ids_to_phases_from_events(
    phases_match=phases_match,
    de_match=de_match,
    enforce_phase_index=False,
    new_col_name="player_possession_event_ids",
    sort_possessions=True
)

# return or print it
print(enriched_phases_match.head())

   index  match_id  frame_start  frame_end time_start time_end  minute_start  \
0      0   1886347           28         89    00:01.8  00:07.9             0   
1      1   1886347           89        107    00:07.9  00:09.7             0   
2      2   1886347          185        232    00:17.5  00:22.2             0   
3      3   1886347          232        283    00:22.2  00:27.3             0   
4      4   1886347          283        301    00:27.3  00:29.1             0   

   second_start  duration  period  attacking_side_id  team_in_possession_id  \
0             1       6.1       1                  1                   1805   
1             7       1.8       1                  1                   1805   
2            17       4.7       1                  2                   4177   
3            22       5.1       1                  2                   4177   
4            27       1.8       1                  1                   1805   

  attacking_side team_in_possession_shortnam

In [13]:
# go through the dynamic events and filter all the events that are of type 'player_possession' and name it pp_all.
pp_all = de_match.loc[de_match["event_type"] == "player_possession"].copy()
pp_all = pp_all.sort_values(["frame_start", "frame_end"]).reset_index(drop=True)
print(f"Filtered {len(pp_all)} player_possession event(s).")

Filtered 999 player_possession event(s).


In [14]:
# Enrich the dynamic events with a 'linked_event' column (only for possession rows)
enriched_de_match = enrich_events_with_linked_events(
    de_match=de_match,
    pp_all=pp_all,
    new_col_name="linked_event",                 # you can rename if you prefer
    child_event_types=("passing_option", "off_ball_run", "on_ball_engagement")  # or set to None to include all
)

# Quick inspection: show possession rows with their linked events
print(
    enriched_de_match.loc[
        enriched_de_match["event_type"] == "player_possession",
        ["event_id", "player_id", "player_name", "frame_start", "frame_end", "linked_event"]
    ].head(10)
)

   event_id  player_id  player_name  frame_start  frame_end  \
0       8_0     966120    B. Gibson           28         28   
1       8_1      51649   A. Šušnjar           48         58   
5       8_2     735578     M. Natta           72         89   
14      8_3      33697  N. Pijnaker          202        229   
19      8_4     133498  F. De Vries          243        251   
24      8_5      50983    D. Ingham          283        299   
31      8_6     965685   L. Gillion          301        337   
39      8_7      23418  F. Gallegos          350        350   
42      8_8      51009     R. Scott          387        430   
43      8_9     735578     M. Natta          451        460   

                       linked_event  
0                                    
1                       7_0,7_1,9_0  
5   7_2,7_3,7_4,7_5,1_0,9_1,1_1,1_2  
14                      7_6,1_3,7_7  
19                 9_2,7_8,7_9,7_10  
24          9_3,7_11,7_12,7_13,7_14  
31   1_4,1_5,7_15,7_16,7_17,9_4,1_6  
39

In [15]:
# de_match: the dynamic events DataFrame for the match
# Add xthreat_at_start (numeric for PP rows; "" for non-PP rows)
enriched_de_match = add_xthreat_at_start_for_possessions(de_match=enriched_de_match, new_col_name="xthreat_at_start")

# Inspect a few rows
print(
    enriched_de_match.loc[
        enriched_de_match['event_type'] == 'player_possession',
        ['event_id','player_id','player_name','end_type','pass_outcome','xthreat_at_start']
    ].head(10)
)

   event_id  player_id  player_name end_type  pass_outcome xthreat_at_start
0       8_0     966120    B. Gibson     pass    successful              0.0
1       8_1      51649   A. Šušnjar     pass    successful              0.0
5       8_2     735578     M. Natta     pass  unsuccessful           0.0001
14      8_3      33697  N. Pijnaker     pass    successful              0.0
19      8_4     133498  F. De Vries     pass  unsuccessful           0.0003
24      8_5      50983    D. Ingham     pass  unsuccessful              0.0
31      8_6     965685   L. Gillion     pass    successful              0.0
39      8_7      23418  F. Gallegos     pass  unsuccessful           0.0345
42      8_8      51009     R. Scott     pass    successful              0.0
43      8_9     735578     M. Natta     pass    successful              0.0


In [16]:
# de_match: your dynamic events DataFrame
enriched_de_match = add_xthreat_at_end_for_possessions(
    de_match=enriched_de_match,
    new_col_name="xthreat_at_end"
)

print(
    enriched_de_match.loc[
        enriched_de_match["event_type"] == "player_possession",
        ["event_id","end_type","pass_outcome","xthreat_at_start","xthreat_at_end"]
    ].head(10)
)

   event_id end_type  pass_outcome xthreat_at_start xthreat_at_end
0       8_0     pass    successful              0.0            0.0
1       8_1     pass    successful              0.0         0.0001
5       8_2     pass  unsuccessful           0.0001            0.0
14      8_3     pass    successful              0.0         0.0003
19      8_4     pass  unsuccessful           0.0003            0.0
24      8_5     pass  unsuccessful              0.0            0.0
31      8_6     pass    successful              0.0         0.0345
39      8_7     pass  unsuccessful           0.0345            0.0
42      8_8     pass    successful              0.0            0.0
43      8_9     pass    successful              0.0            0.0


In [17]:
# enriched_de_match: your current events DataFrame (already enriched with `linked_event`,
#                    and with xthreat/xpass_completion populated on passing_option rows)

enriched_de_match = add_xthreat_potential_max(
    enriched_de_match=enriched_de_match,
    linked_col="linked_event",              # if you named it differently, change here
    new_col_name="xthreat_potential_max"
)

# Sanity check: PP rows should have float values; other rows should be empty ""
print(
    enriched_de_match.loc[
        enriched_de_match["event_type"] == "player_possession",
        ["event_id","xthreat_at_start","xthreat_at_end","xthreat_potential_max"]
    ].head(12)
)

   event_id xthreat_at_start xthreat_at_end xthreat_potential_max
0       8_0              0.0            0.0                   0.0
1       8_1              0.0         0.0001              0.000578
5       8_2           0.0001            0.0              0.005807
14      8_3              0.0         0.0003              0.001151
19      8_4           0.0003            0.0              0.003398
24      8_5              0.0            0.0              0.000131
31      8_6              0.0         0.0345              0.041567
39      8_7           0.0345            0.0              0.016532
42      8_8              0.0            0.0                   0.0
43      8_9              0.0            0.0              0.001345
46     8_10              0.0            0.0               0.00132
53     8_11              0.0            0.0                   0.0


In [18]:
# Starting from your events DF after you've already added:
#  - xthreat_at_start
#  - xthreat_at_end
#  - xthreat_potential_max

enriched_de_match = add_xthreat_deltas(
    enriched_de_match=enriched_de_match,
    col_start="xthreat_at_start",
    col_end="xthreat_at_end",
    col_pmax="xthreat_potential_max",
    out_delta="xthreat_increase",
    out_pot_red="potential_xthreat_reduction"
)

# Inspect a few player possession rows:
print(
    enriched_de_match.loc[
        enriched_de_match["event_type"] == "player_possession",
        ["event_id","xthreat_at_start","xthreat_at_end","xthreat_potential_max",
        "xthreat_increase","potential_xthreat_reduction"]
    ].head(12)
)

   event_id xthreat_at_start xthreat_at_end xthreat_potential_max  \
0       8_0              0.0            0.0                   0.0   
1       8_1              0.0         0.0001              0.000578   
5       8_2           0.0001            0.0              0.005807   
14      8_3              0.0         0.0003              0.001151   
19      8_4           0.0003            0.0              0.003398   
24      8_5              0.0            0.0              0.000131   
31      8_6              0.0         0.0345              0.041567   
39      8_7           0.0345            0.0              0.016532   
42      8_8              0.0            0.0                   0.0   
43      8_9              0.0            0.0              0.001345   
46     8_10              0.0            0.0               0.00132   
53     8_11              0.0            0.0                   0.0   

   xthreat_increase potential_xthreat_reduction  
0               0.0                         0.0  
1 

In [19]:
# enriched_de_match: your current events DF after previous enrichments
# enriched_tracking_data: per-frame tracking (columns: frame, player_id, x, y, is_detected, [ball_status])

# If 'team_id' is present and correct in enriched_de_match, you can omit team_map:
enriched_de_match = add_passing_option_proximity(
    enriched_de_match=enriched_de_match,
    enriched_tracking_data=enriched_tracking_data,
    new_col_name="po_proximity_score",   # you can rename as you like
    team_map=None,                       # or pass a DataFrame ['player_id','team_id'] to control mapping
    require_ball_in_play=False           # set True if you want to ignore dead-ball frames
)

# Quick sanity check
print(
    enriched_de_match.loc[
        enriched_de_match["event_type"] == "passing_option",
        ["event_id","player_id","frame_start","frame_end","po_proximity_score"]
    ].head(12)
)

   event_id  player_id  frame_start  frame_end po_proximity_score
2       7_0     735574           48         53           0.842568
3       7_1     735578           48         58          15.118051
6       7_2      50978           72         82           7.542031
7       7_3     795507           72         89           6.150422
8       7_4     966120           72         89           4.566894
9       7_5     735573           73         89           6.190666
15      7_6     133498          202        229          11.022776
17      7_7     965685          228        229                NaN
20      7_8      33697          243        246           9.263544
21      7_9      23418          243        250           4.949257
22     7_10     965685          243        251           6.589251
25     7_11      51009          283        290          21.992765


In [20]:
# enriched_de_match: your enriched events DataFrame (with player_possession & passing_option rows, etc.)
# enriched_tracking_data: per-frame tracking with columns: frame, player_id, x, y, is_detected, [ball_status]

# If `team_id` is present and correct in enriched_de_match, you can omit team_map:
enriched_de_match = add_possession_proximity(
    enriched_de_match=enriched_de_match,
    enriched_tracking_data=enriched_tracking_data,
    new_col_name="pp_proximity_score",
    team_map=None,                 # or pass a DataFrame ['player_id','team_id'] if you prefer to control mapping
    require_ball_in_play=False     # set True to ignore frames where ball_status == 'dead'
)

# Inspect a few player possession rows:
print(
    enriched_de_match.loc[
        enriched_de_match["event_type"] == "player_possession",
        ["event_id","player_id","frame_start","frame_end","pp_proximity_score"]
    ].head(12)
)

   event_id  player_id  frame_start  frame_end pp_proximity_score
0       8_0     966120           28         28           8.657043
1       8_1      51649           48         58          11.245402
5       8_2     735578           72         89           4.748693
14      8_3      33697          202        229          11.421757
19      8_4     133498          243        251           4.258072
24      8_5      50983          283        299           1.871307
31      8_6     965685          301        337            2.30156
39      8_7      23418          350        350           3.421871
42      8_8      51009          387        430           8.323816
43      8_9     735578          451        460          14.483534
46     8_10      51649          469        504          10.272907
53     8_11      51713          526        526           4.806079


In [21]:
# enriched_de_match: your current events DF (already has `linked_event`
# and `po_proximity_score` computed for passing_option rows)

enriched_de_match = add_avg_po_proximity_score(
    enriched_de_match=enriched_de_match,
    linked_col="linked_event",             # change only if your linked column has a different name
    po_score_col="po_proximity_score",
    new_col_name="avg_po_proximity_score"
)

# Quick sanity check: PP rows should have a float or NaN; others should be empty
print(
    enriched_de_match.loc[
        enriched_de_match["event_type"] == "player_possession",
        ["event_id","avg_po_proximity_score"]
    ].head(12)
)

   event_id avg_po_proximity_score
0       8_0                    NaN
1       8_1               7.980309
5       8_2               6.112504
14      8_3              11.022776
19      8_4               6.934017
24      8_5              11.683156
31      8_6                3.53588
39      8_7               4.635807
42      8_8                    NaN
43      8_9              13.583588
46     8_10              12.788752
53     8_11                    NaN


## Notes
- This notebook keeps code minimal and imports reusable helpers from `src/`.
- Adjust `match_id` or phase selection as needed for specific analyses.